In [1]:
import json
import tqdm
import re
import pandas as pd
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [2]:
keyTestTPath='data/key_test_t.csv'

In [ ]:
def divideFile(output_dir = 'data/small_files'):
    # 判断输出目录下是否存在文件，如果存在则删除
    if os.path.exists(output_dir):
        for file in os.listdir(output_dir):
            os.remove(os.path.join(output_dir, file))
    else:
        os.makedirs(output_dir, exist_ok=True)

    num_files = 100
    # 列名或列索引列表，只保留需要的列
    keep_columns = ['results', 'dimension']

    os.makedirs(output_dir, exist_ok=True)

    df = pd.read_csv(keyTestTPath)

    chunk_size = len(df) // num_files

    for i in range(num_files):
        # 计算当前小文件的起始行和结束行
        start = i * chunk_size
        end = (i + 1) * chunk_size if i < num_files - 1 else len(df)
        
        # 提取当前小文件的数据
        small_df = df.iloc[start:end, :][keep_columns]
        
        # 将当前小文件保存为CSV文件
        file_name = f'small_file_{i + 1}.csv'
        file_path = os.path.join(output_dir, file_name)
        small_df.to_csv(file_path, index=False)
        if i % 10 == 0:
            print(f'Small file {file_name} has been saved.')

    print('All files have been saved.')


In [ ]:
divideFile()

In [ ]:
import glob
import concurrent.futures
from concurrent.futures import ThreadPoolExecutor

def judgeJson(keyTestT):
    for i in range(len(keyTestT)):
        try:
            json.loads(keyTestT.loc[i, 'dimension'])
            json.loads(keyTestT.loc[i, 'results'])
        except:
            keyTestT.drop(i, inplace=True)
    keyTestT = keyTestT.reset_index(drop=True)
    return keyTestT


def filterDataByKey(keyTestT, keyName):
    correctNum=0
    for i in range(len(keyTestT)):
        results = json.loads(keyTestT.loc[i, 'results'])
        # 将resuls转为字符串，使用正则表达式判断是否包含keyName
        resultsStr = str(results)
        if re.search(keyName, resultsStr):
            correctNum+=1
    return correctNum


def getResultArgList(keyTestT):
    resultArgList = []
    for i in range(len(keyTestT)):
        results = json.loads(keyTestT.loc[i, 'results'])
        # 将resuls转为字符串，使用正则表达式判断是否包含#字符，若包含则将#字符串提取出来
        resultsStr = str(results)
        resultArg = re.findall(r'#\w+', resultsStr)
        resultArgList.extend(resultArg)
    # 去重
    resultArgList = list(set(resultArgList))
    print('arg num:', len(resultArgList))
    # print('five for example:', resultArgList[:5])
    return keyTestT,resultArgList


def getNumOfKey(keyTestT, resultArgList):
    keyNumList = []
    # 调用filterDataByKey函数，统计每个key的数量
    for key in resultArgList:
        keyNumList.append(filterDataByKey(keyTestT, key))
    return keyNumList


# 文件夹路径
smallFilePath = 'data/small_files/'
# 文件名的通配符
file_pattern = 'small_file_*.csv'
# 获取文件夹下所有符合通配符的文件路径
file_paths = glob.glob(smallFilePath + file_pattern)

# 定义处理单个文件的函数
def process_csv_file(file_path):
    # 读取当前文件的数据
    df = pd.read_csv(file_path)
    df = judgeJson(df)
    df, resultArgList = getResultArgList(df)
    keyNumList = getNumOfKey(df, resultArgList)
    keyNumDict = dict(zip(resultArgList, keyNumList))
    return df, keyNumDict

# 使用ThreadPoolExecutor并行处理多个文件
with concurrent.futures.ThreadPoolExecutor(max_workers=15) as executor:
    futures = {executor.submit(process_csv_file, file_path): file_path for file_path in file_paths}
    combined_keyNumDict = {}
    dfs = []
    for future in concurrent.futures.as_completed(futures):
        try:
            df, keyNumDict = future.result()
            combined_keyNumDict.update(keyNumDict)
            dfs.append(df)
        except Exception as e:
            print(f'Error processing file: {futures[future]}, {e}')


df_combined = pd.concat(dfs, ignore_index=True)

df_combined.to_csv('data/combined_small_files.csv', index=False)

print(combined_keyNumDict)

In [ ]:
import os
import time
import pandas as pd

small_file_path = 'data/small_files/'

# 定义处理单个文件的函数
def process_csv_file(file_path):
    # 读取当前文件的数据
    df = pd.read_csv(file_path)
    df = judgeJson(df)
    df, resultArgList = getResultArgList(df)
    keyNumList = getNumOfKey(df, resultArgList)
    keyNumDict = dict(zip(resultArgList, keyNumList))
    return df, keyNumDict

# 获取文件夹下所有文件的路径
file_paths = [os.path.join(small_file_path, file) for file in os.listdir(small_file_path) if file.endswith('.csv')]

# 顺序执行文件，并在处理完每个文件后合并 keyNumDict
combined_keyNumDict = {}
dfs = []
for file_path in file_paths:
    df, keyNumDict = process_csv_file(file_path)
    combined_keyNumDict.update(keyNumDict)
    dfs.append(df)

# 将所有小文件的结果合并为一个 DataFrame
df_combined = pd.concat(dfs, ignore_index=True)

# 将合并后的结果保存为一个 CSV 文件
df_combined.to_csv('data/combined_small_files.csv', index=False)

# 打印合并后的总字典
print(combined_keyNumDict)

In [3]:
keyTestT=pd.read_csv(keyTestTPath,usecols=['results','dimension','results_key'])

In [4]:
def judgeJson(keyTestT):
    for i in tqdm.tqdm(range(len(keyTestT))):
        try:
            json.loads(keyTestT.loc[i, 'dimension'])
            json.loads(keyTestT.loc[i, 'results'])
        except:
            keyTestT.drop(i, inplace=True)
    keyTestT = keyTestT.reset_index(drop=True)
    return keyTestT

In [ ]:
keyTestT=judgeJson(keyTestT)

In [ ]:
keyTestT.info()

In [ ]:
# def filterDataByKey(keyTestT, keyName):
#     correctNum=0
#     for i in range(len(keyTestT)):
#         results = json.loads(keyTestT.loc[i, 'results'])
#         for result in results:
#             if keyName in result:
#                 correctNum+=1
#                 break
#     return correctNum


def filterDataByKey(keyTestT, keyName):
    correctNum=0
    for i in range(len(keyTestT)):
        results = json.loads(keyTestT.loc[i, 'results'])
        # 将resuls转为字符串，使用正则表达式判断是否包含keyName
        resultsStr = str(results)
        if re.search(keyName, resultsStr):
            correctNum+=1
    return correctNum


def getResultArgList(keyTestT,resultArgList):
    for index, row in tqdm.tqdm(keyTestT.iterrows(), total=keyTestT.shape[0]):
        result = row['results']
        resultJson = json.loads(result)
        for key in resultJson:
            if key.find('#') != -1:
                resultArgList.append(key)

    resultArgList = list(set(resultArgList))
    print('arg num:', len(resultArgList))
    print('five for example:', resultArgList[:5])
    return keyTestT,resultArgList


def getNumOfKey(keyTestT, resultArgList):
    keyNumList = []
    # 调用filterDataByKey函数，统计每个key的数量
    for key in resultArgList:
        keyNumList.append(filterDataByKey(keyTestT, key))
    return keyNumList


In [ ]:
resultArgList=[]

In [ ]:
keyTestT, resultArgList = getResultArgList(keyTestT,resultArgList)

In [ ]:
keyNumList = getNumOfKey(keyTestT, resultArgList)

In [ ]:
def showInfoList(yourlist):
    # 展示列表的信息
    print('max:', max(yourlist))
    print('min:', min(yourlist))
    print('median:', np.median(yourlist))
    print('len:', len(yourlist))
    return max(yourlist), min(yourlist), np.median(yourlist)

def getTreshold(yourlist):
    # 获取阈值
    yourMax, yourMin, youMdian = showInfoList(yourlist)
    diff = yourMax - youMdian
    thresholds = [youMdian + 0.1 * diff, youMdian + 0.2 * diff, youMdian + 0.3 * diff]
    print("\nPossible thresholds:")
    for threshold in thresholds:
        print(threshold)
    return thresholds

In [ ]:
treshold=getTreshold(keyNumList)

In [ ]:
# 画图
plt.figure(figsize=(90, 20))
plt.bar(range(len(keyNumList)), keyNumList)
plt.xticks(range(len(keyNumList)), resultArgList, rotation=90)
plt.show()

In [ ]:
def getMostImportantKey(resultArgList, keyNumList, threshold):
    # 获取重要的key
    importantKeyList = []
    for i in range(len(keyNumList)):
        if keyNumList[i] > threshold:
            importantKeyList.append(resultArgList[i])
    return importantKeyList

In [ ]:
importantKeyList = getMostImportantKey(resultArgList, keyNumList, max(treshold))

In [ ]:
def getImportantKeyDict(resultArgList,keyNumList,importantKeyList):
    # 获取重要key的字典
    importantKeyDict = {}
    for i in range(len(importantKeyList)):
        importantKeyDict[importantKeyList[i]] = keyNumList[resultArgList.index(importantKeyList[i])]
    return importantKeyDict

def showDict(yourDict):
    # 展示字典
    for key in yourDict:
        print(key, ':', yourDict[key])

In [ ]:
importantKeyDict=getImportantKeyDict(resultArgList,keyNumList,importantKeyList)

In [ ]:
showDict(importantKeyDict)

In [ ]:
def getValueByKey(keyTestT, key):
    valueList=[]
    keyTestT = keyTestT.reset_index(drop=True)
    for i in tqdm.tqdm(range(len(keyTestT))):
        results = json.loads(keyTestT.loc[i, 'results'])
        for result in results:
            if key in result:
                try:
                    valueList.append(results[key])
                except:
                    print('error:', key, i)
                    # print(results[key])
                    valueList.append(0)                
                break
    return valueList

In [ ]:
# 获得每一个重要key的值
def getAllValues(keyTestT, importantKeyList):
    valuesDict = {}
    for key in importantKeyList:
        valuesDict[key] = getValueByKey(keyTestT, key)
    return valuesDict

In [ ]:
valuesDict = getAllValues(keyTestT, importantKeyList)

In [ ]:
showDict(valuesDict)

In [ ]:
# 将valuesDict中的#stream_copy，#stream_triad提取出来，组成DataFrame
streamCopyTriad = pd.DataFrame()
streamCopyTriad['stream_copy'] = valuesDict['#stream_copy']
streamCopyTriad['stream_triad'] = valuesDict['#stream_triad']
streamCopyTriad.shape

In [ ]:
streamCopyTriad.head()

In [ ]:
def getInput(keyTestT, importantKeyList):
    # 根据result中是否有importantKeyList[1]和importantKeyList[3]来提取dimension输入
    inputList = []
    keys_to_extract = ['cvm_cpu', 'cvm_memory', 'cvm_cpu_qos', 'cvm_os_type']
    for i in tqdm.tqdm(range(len(keyTestT))):
        results = json.loads(keyTestT.loc[i, 'results'])
        has_key1 = False
        has_key2 = False
        for result in results:
            if importantKeyList[3] in result:
                has_key1 = True
            if importantKeyList[5] in result:
                has_key2 = True
            if has_key1 == True and has_key2 == True:
                # 将dimension中['cvm_cpu', 'cvm_memory', 'cvm_cpu_qos', 'cvm_os_type']这几个字段的值提取出来
                dimension = json.loads(keyTestT.loc[i, 'dimension'])
                dimension = json.dumps(dimension)
                templist=['cvm_cpu', 'cvm_memory', 'cvm_cpu_qos', 'cvm_os_type']
                dimensionJson = json.loads(dimension)
                for key in dimensionJson:
                    if key in templist:
                        key_value = dimensionJson[key]
                        if key=='cvm_memory':
                            key_value=float(key_value.split(' ')[0])
                        elif key=='cvm_cpu':
                            key_value=float(key_value)
                        elif key=='cvm_cpu_qos':
                            key_value='true'
                        templist[keys_to_extract.index(key)]=key_value
                inputList.append(templist)
                break
    return inputList

In [ ]:
inputList = getInput(keyTestT, importantKeyList)

In [ ]:
print(len(inputList))
# 打印前5个
for i in range(5):
    print(inputList[i])

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [ ]:
def showPic(methodName, y_test, y_pred):
    # 创建柱状图
    plt.figure(figsize=(80, 15))

    plt.subplot(1, 3, 1)
    plt.scatter(y_test, y_pred)
    plt.xlabel('True Values')
    plt.ylabel('Predicted Values')
    plt.title('{}: True Values vs. Predicted Values'.format(methodName))

    plt.subplot(1, 3, 2)
    # 创建样本索引数组
    index = np.arange(len(y_test))
    # 绘制线性回归模型的折线图
    plt.plot(index, y_test, label='True Values')
    plt.plot(index, y_pred, label='Predicted Values')
    plt.xlabel('Sample Index')
    plt.ylabel('Values')
    plt.title('{}: True Values vs. Predicted Values'.format(methodName))
    plt.legend()

    plt.subplot(1, 3, 3)
    # 计算残差
    residuals = y_test - y_pred
    # 绘制线性回归模型的残差图
    plt.scatter(y_test, residuals)
    plt.xlabel('True Values')
    plt.ylabel('Residuals')
    plt.title('{}: True Values vs. Residuals'.format(methodName))
    plt.axhline(0, color='red', linestyle='--')

    plt.show()



def train(X, y):
    # 划分训练集和测试集
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

    # 初始化模型
    linear_reg = LinearRegression()
    decision_tree_reg = DecisionTreeRegressor()
    random_forest_reg = RandomForestRegressor(n_estimators=10000, max_depth=5)
    svm_reg = SVR(C=1.0, epsilon=0.1)
    knn_reg = KNeighborsRegressor(n_neighbors=10, weights='uniform')


    # 拟合模型
    linear_reg.fit(X_train, y_train)
    decision_tree_reg.fit(X_train, y_train)
    random_forest_reg.fit(X_train, y_train)
    svm_reg.fit(X_train, y_train)
    knn_reg.fit(X_train, y_train)


    # 预测
    y_pred_linear_reg = linear_reg.predict(X_test)
    y_pred_decision_tree_reg = decision_tree_reg.predict(X_test)
    y_pred_random_forest_reg = random_forest_reg.predict(X_test)
    y_pred_svm_reg = svm_reg.predict(X_test)
    y_pred_knn_reg = knn_reg.predict(X_test)


    # 计算评估指标
    mse_linear_reg = mean_squared_error(y_test, y_pred_linear_reg)
    mse_decision_tree_reg = mean_squared_error(y_test, y_pred_decision_tree_reg)
    mse_random_forest_reg = mean_squared_error(y_test, y_pred_random_forest_reg)
    mse_svm_reg = mean_squared_error(y_test, y_pred_svm_reg)
    mse_knn_reg = mean_squared_error(y_test, y_pred_knn_reg)

    mae_linear_reg = mean_absolute_error(y_test, y_pred_linear_reg)
    mae_decision_tree_reg = mean_absolute_error(y_test, y_pred_decision_tree_reg)
    mae_random_forest_reg = mean_absolute_error(y_test, y_pred_random_forest_reg)
    mae_svm_reg = mean_absolute_error(y_test, y_pred_svm_reg)
    mae_knn_reg = mean_absolute_error(y_test, y_pred_knn_reg)

    r2_linear_reg = r2_score(y_test, y_pred_linear_reg)
    r2_decision_tree_reg = r2_score(y_test, y_pred_decision_tree_reg)
    r2_random_forest_reg = r2_score(y_test, y_pred_random_forest_reg)
    r2_svm_reg = r2_score(y_test, y_pred_svm_reg)
    r2_knn_reg = r2_score(y_test, y_pred_knn_reg)


    # 打印评估指标
    # print('Linear Regression MSE:', mse_linear_reg)
    # print('Decision Tree Regression MSE:', mse_decision_tree_reg)
    # print('Random Forest Regression MSE:', mse_random_forest_reg)
    # print('Support Vector Regression MSE:', mse_svm_reg)
    # print('KNN Regression MSE:', mse_knn_reg)
    # print('\n')
    # print('Linear Regression MAE:', mae_linear_reg)
    # print('Decision Tree Regression MAE:', mae_decision_tree_reg)
    # print('Random Forest Regression MAE:', mae_random_forest_reg)
    # print('Support Vector Regression MAE:', mae_svm_reg)
    # print('KNN Regression MAE:', mae_knn_reg)
    # print('\n')
    print('Linear Regression R2:', r2_linear_reg)
    print('Decision Tree Regression R2:', r2_decision_tree_reg)
    print('Random Forest Regression R2:', r2_random_forest_reg)
    print('Support Vector Regression R2:', r2_svm_reg)
    print('KNN Regression R2:', r2_knn_reg)
    print('\n')

    models = ['Linear Regression', 'Decision Tree Regression', 'Random Forest Regression', 'SVM Regression', 'KNN Regression']
    y_pred = ['y_pred_linear_reg', 'y_pred_decision_tree_reg', 'y_pred_random_forest_reg', 'y_pred_svm_reg', 'y_pred_knn_reg']
    for i in range(len(models)):
        showPic(models[i], y_test, eval(y_pred[i]))


In [ ]:
df = pd.DataFrame(inputList, columns=['cvm_cpu', 'cvm_memory', 'cvm_cpu_qos', 'cvm_os_type'])
# 判断['cvm_cpu', 'cvm_memory']这两列的值是否是数字，若不是数字则替换为0
df[['cvm_cpu', 'cvm_memory']] = df[['cvm_cpu', 'cvm_memory']].apply(pd.to_numeric, errors='coerce').fillna(0)
# 判断['cvm_cpu_qos', 'cvm_os_type']这两列的值是否是字符串，若不是字符串则转换为字符串
df[['cvm_cpu_qos', 'cvm_os_type']] = df[['cvm_cpu_qos', 'cvm_os_type']].astype(str)

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
from lightgbm import LGBMRegressor

In [ ]:
df['cvm_cpu'] = pd.to_numeric(df['cvm_cpu'])
df['cvm_memory'] = pd.to_numeric(df['cvm_memory'])
one_hot_df = pd.get_dummies(df, columns=['cvm_cpu_qos', 'cvm_os_type'])
X = one_hot_df
y = streamCopyTriad['stream_copy']

# category_features = [x for x in df.columns if df[x].dtype != np.float]
# for i in category_features:
#     df[i] = df[i].astype('category')

train(X, y)

# 以下代码暂停使用

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(df, y , test_size=0.2, random_state=42)

# # 创建并训练 LGBMRegressor 模型
# lgbm_reg = LGBMRegressor(n_estimators=10000)
# lgbm_reg.fit(X_train, y_train)

# # 进行预测
# y_pred = lgbm_reg.predict(X_test)

# # 计算评估指标
# mse = mean_squared_error(y_test, y_pred)
# print("Mean Squared Error:", mse)
# mae = mean_absolute_error(y_test, y_pred)
# print("Mean Absolute Error:", mae)
# r2 = r2_score(y_test, y_pred)
# print("R2 Score:", r2)

In [ ]:
# # one-hot编码
# df = pd.DataFrame(inputList, columns=['cvm_cpu', 'cvm_memory', 'cvm_cpu_qos', 'cvm_os_type'])
# one_hot_df = pd.get_dummies(df)
# # print(one_hot_df)
# print(one_hot_df.shape)
# # 将one-hot编码后的数据和streamCopyTriad合并
# # one_hot_data = pd.concat([one_hot_df, streamCopyTriad], axis=1)
# # print(one_hot_data.shape)

# X = one_hot_df
# y = streamCopyTriad['stream_copy']

# train(X, y)

In [ ]:
# from sklearn.preprocessing import LabelEncoder

# encoder = LabelEncoder()
# df['cvm_cpu_qos'] = encoder.fit_transform(df['cvm_cpu_qos'])
# df['cvm_os_type'] = encoder.fit_transform(df['cvm_os_type'])
# # 合并编码后的列与前两列
# merged_label_df = df[['cvm_cpu', 'cvm_memory', 'cvm_cpu_qos', 'cvm_os_type']]
# print(merged_label_df.shape)
# dfL = df.astype(str)
# merged_label_df = dfL.apply(encoder.fit_transform)
# # 将label编码后的数据和streamCopyTriad合并
# # label_data = pd.concat([merged_label_df, streamCopyTriad], axis=1)
# # print(label_data.shape)
# # label_data.head()

# X = merged_label_df
# y = streamCopyTriad['stream_copy']

# # train(X, y)

In [ ]:
# # 二进制编码
# binary_encoded_df = pd.get_dummies(df, prefix='', prefix_sep='')
# # print(binary_encoded_df)
# print(binary_encoded_df.shape)
# # 将二进制编码后的数据和streamCopyTriad合并
# # binary_data = pd.concat([binary_encoded_df, streamCopyTriad], axis=1)
# # print(binary_data.shape)

# X = binary_encoded_df
# y = streamCopyTriad['stream_copy']

# # train(X, y)

In [ ]:
# # 计数count编码
# count_encoded_df = df.replace(df.stack().value_counts(normalize=True).to_dict())
# # print(count_encoded_df)
# print(count_encoded_df.shape)
# # 将计数编码后的数据和streamCopyTriad合并
# # count_data = pd.concat([count_encoded_df, streamCopyTriad], axis=1)
# # print(count_data.shape)
# # count_data.head()

# X = count_encoded_df
# y = streamCopyTriad['stream_copy']

# # train(X, y)